In [1]:
from kedro.config import ConfigLoader
import yaml
import os
import pandas as pd
from kedro.io import DataCatalog
from kedro_datasets.pandas import (CSVDataSet)
from kedro.framework.context import KedroContext

In [2]:
catalog.list()

['p18_19_0',
 'p18_19_1',
 'p18_19_3',
 'p18_19_5',
 'qld_p18_19',
 'qld_all',
 'parameters']

In [3]:
df = catalog.load('qld_p18_19')

sheet_name = "p18_19"

2023-04-25 17:56:38,439 - kedro.io.data_catalog - INFO - Loading data from 'qld_p18_19' (ExcelDataSet)...


In [4]:
df

,0,1,2,3,4,5
0,Number of FTE and percentage by gender and agency,NaN,NaN,NaN,NaN,NaN
1,NaN,FTE,NaN,NaN,Percentage,NaN
2,Agency,Female,Male,Total,Female,Male
3,Department of Aboriginal and Torres Strait Isl...,196.04,88.8,284.84,0.688246,0.311754
4,Department of Agriculture and Fisheries,862.92,1198.25,2061.17,0.418655,0.581345
...,...,...,...,...,...,...
89,Resources Safety and Health Queensland,120,176,296,0.405405,0.594595
90,State Library of Queensland,255,98,353,0.72238,0.27762
91,Trade and Investment Queensland,83,52,135,0.614815,0.385185
92,Sector sub-total: Other entities,1582,778,2360,0.670339,0.329661


In [5]:
empty_row_index = df.index[df.isnull().all(axis=1)].insert(0,0)
empty_row_index

Int64Index([0, 32, 46, 47, 80], dtype='int64')

In [6]:
dataframes = {}

for i in range(len(empty_row_index)-1):
    df_name = f"{sheet_name}_{i}"

    # Skip adjacent empty rows
    if empty_row_index[i+1]-empty_row_index[i] == 1:
        continue

    # Extract other tables
    df_value = df.iloc[empty_row_index[i]:empty_row_index[i+1]].dropna(how='all').reset_index(drop=True)

    # Add the dataframe into the library
    dataframes[df_name] = df_value

# Extract the last table
df_value = df.iloc[empty_row_index[-1]:df.index[-1]+1].dropna(how='all').reset_index(drop=True)
dataframes[f"{sheet_name}_{len(empty_row_index)}"] = df_value

In [7]:
dataframes



{'p18_19_0':                                                     0          1         2  \
 0   Number of FTE and percentage by gender and agency        NaN       NaN   
 1                                                 NaN        FTE       NaN   
 2                                              Agency     Female      Male   
 3   Department of Aboriginal and Torres Strait Isl...     196.04      88.8   
 4             Department of Agriculture and Fisheries     862.92   1198.25   
 5         Department of Child Safety, Youth and Women    2748.27    552.54   
 6   Department of Communities, Disability Services...    1158.26     576.8   
 7                             Department of Education   57370.35  17224.97   
 8   Department of Employment, Small Business and T...     396.41    162.06   
 9               Department of Environment and Science     1393.8   1498.85   
 10             Department of Housing and Public Works    2880.64      2361   
 11         Department of Justice and At

In [8]:
for name, df in dataframes.items():

    filepath = f"../data/02_extracted/{name}.csv"

    csv_data_set = CSVDataSet(filepath)

    # Save the DataFrame to the CSV file using the save() method of the CSVDataSet object
    csv_data_set.save(df)

    # Add the CSVDataSet object to the catalog
    catalog.add(name, csv_data_set)

    # Update the catalog yml file
    catalog.save(name=name, data=catalog.load(name))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:11                                                                             │
│                                                                                                  │
│    8 │   csv_data_set.save(df)                                                                   │
│    9 │                                                                                           │
│   10 │   # Add the CSVDataSet object to the catalog                                              │
│ ❱ 11 │   catalog.add(name, csv_data_set)                                                         │
│   12 │                                                                                           │
│   13 │   # Update the catalog yml file                                                           │
│   14 │   catalog.save(name=name, data=catalog.load(name))                                        │
│                                                                                                  │
│ C:\Users\gdbt0\anaconda3\envs\Data-preparation-pipeline\lib\site-packages\kedro\io\data_catalog. │
│ py:449 in add                                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
DataSetAlreadyExistsError: DataSet 'p18_19_0' has already been registered

In [ ]:
# Save the catalog changes to catalog.yml
catalog.list()

In [ ]:
# Load the catalog YAML file

# Define the source of configuration files
conf_path = "..\conf"
conf_loader = ConfigLoader(conf_source=conf_path, env="local")
conf_catalog = conf_loader.get("catalog.yml")

# Define the filename and path relative to the project root directory
#filename = "catalog.yml"
#path = os.path.join(os.getcwd(), filename)

path = "C:/Users/gdbt0/Projects/Data-preparation-pipeline/data-processing-pipeline/conf/base/catalog.yml"

for name in dataframes.keys():
    # Add a new dataset to the catalog YAML file
    conf_catalog[name] = {
        "type": "pandas.CSVDataSet",
        "filepath": f"data/02_extracted/{name}.csv"
    }
    conf_loader.update()

    # Save the updated catalog YAML file
    with open(path, "w") as f:
       yaml.dump(conf_catalog, f)

In [ ]:
catalog.list()